In [1]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
#https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


import keras
import os
from shutil import copyfile
import random
import matplotlib.pyplot as plt
import numpy as np


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [38]:
print ("keras.__version__:", keras.__version__)
fig = plt.figure()
size = fig.get_size_inches()
print ("size:", size, "fig.dpi:", fig.dpi)

keras.__version__: 2.0.8
size: [ 6.  4.] fig.dpi: 72.0


In [3]:
# dimensions of our images.
img_width, img_height = 150, 150

In [9]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]


In [4]:
categories = ['cats', 'dogs']
DATA_BASE = "/home/ubuntu/fastai/data/dogscats/"
SAMPLE_TRAIN = DATA_BASE + "sample/train/"
SAMPLE_VALID = DATA_BASE + "sample/valid/"
TRAIN        = DATA_BASE + "train/"
VALID        = DATA_BASE + "valid/"
RESULTS = DATA_BASE + "models/"
SAMPLESIZE = 1000

In [13]:
dirList = [SAMPLE_TRAIN, SAMPLE_VALID, TRAIN, VALID]

for dir_path in dirList:
    for category in categories:
        print ("# of files in", dir_path+category, ":", len(dirFileList(dir_path+category)))

for dir_path in [SAMPLE_TRAIN, SAMPLE_VALID, TRAIN, VALID]:
    print ("# of files in", dir_path, ":", len(dirFileList(dir_path)))

# of files in /home/ubuntu/fastai/data/dogscats/sample/train/cats : 1000
# of files in /home/ubuntu/fastai/data/dogscats/sample/train/dogs : 1000
# of files in /home/ubuntu/fastai/data/dogscats/sample/valid/cats : 1000
# of files in /home/ubuntu/fastai/data/dogscats/sample/valid/dogs : 1000
# of files in /home/ubuntu/fastai/data/dogscats/train/cats : 11500
# of files in /home/ubuntu/fastai/data/dogscats/train/dogs : 11500
# of files in /home/ubuntu/fastai/data/dogscats/valid/cats : 1004
# of files in /home/ubuntu/fastai/data/dogscats/valid/dogs : 1004
# of files in /home/ubuntu/fastai/data/dogscats/sample/train/ : 0
# of files in /home/ubuntu/fastai/data/dogscats/sample/valid/ : 0
# of files in /home/ubuntu/fastai/data/dogscats/train/ : 0
# of files in /home/ubuntu/fastai/data/dogscats/valid/ : 0


In [15]:
top_model_weights_path = RESULTS+'bottleneck_fc_model.h5'
train_data_dir = SAMPLE_TRAIN
validation_data_dir = SAMPLE_VALID
nb_train_samples = len(dirFileList(SAMPLE_TRAIN+'cats/')) + len(dirFileList(SAMPLE_TRAIN+'dogs/'))#  2000
nb_validation_samples = len(dirFileList(SAMPLE_VALID+'cats/')) + len(dirFileList(SAMPLE_VALID+'dogs/'))
print ("nb_train_samples:", nb_train_samples)
print ("nb_validation_samples:", nb_validation_samples)
epochs = 50
batch_size = 16

nb_train_samples: 2000
nb_validation_samples: 2000


In [35]:
print ("nb_train_samples // batch_size = ", nb_train_samples // batch_size)
print ("top_model_weights_path:", top_model_weights_path)

nb_train_samples // batch_size =  125
top_model_weights_path: /home/ubuntu/fastai/data/dogscats/models/bottleneck_fc_model.h5


In [29]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
print ("type(bottleneck_features_train):", type(bottleneck_features_train))

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
print ("type(bottleneck_features_validation):", type(bottleneck_features_validation))


Found 2000 images belonging to 2 classes.
type(bottleneck_features_train): <class 'numpy.ndarray'>
Found 2000 images belonging to 2 classes.
type(bottleneck_features_validation): <class 'numpy.ndarray'>


In [32]:
print (type(nb_train_samples))
(nb_train_samples / 2)

<class 'int'>


1000.0

In [36]:
train_data = bottleneck_features_train
train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

validation_data = bottleneck_features_validation
validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

print (model.summary())

history = model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________
None
Train on 2000 samples, validate on 2000 samples
Epoch 1/50
2000/2000 [==============================] - 1s - loss: 0.6908 - acc: 0.7565 - val_loss: 0.4513 - val_acc: 0.7985
Epoch 2/50
2000/2000 [==============================] - 1s - loss: 0.3828 - acc: 0.836

In [37]:
print(history.history.keys()) 

dict_keys(['acc', 'val_loss', 'loss', 'val_acc'])


In [ ]:
plt.figure(1, figsize=(20,10))  
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()  
